In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [2]:
%cd /gdrive/MyDrive/nlp-yuan_code/FinBERT-QA

/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA


In [3]:
from statistics import mean 

from src.process_data import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
!pip install pyserini
!pip install faiss

!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libomp-dev is already the newest version (5.0.1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#from pyserini.search import SimpleSearcher

import pandas as pd
import regex as re
import csv
from itertools import islice
import pickle
import numpy as np
import json
import os
import sys
import argparse
from pathlib import Path
from sklearn.model_selection import train_test_split
from pathlib import Path
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#from pyserini.search as pysearch

from pyserini.search import SimpleSearcher

from src.utils import *


In [6]:
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install pydot --quiet

In [7]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from transformers import BertTokenizer, TFBertModel


import sklearn as sk
import nltk
from nltk.corpus import reuters
from nltk.data import find

import matplotlib.pyplot as plt

import re

In [8]:
data_path = '/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/'
# Document id and Answer text
collection = load_answers_to_df(data_path+"FiQA_train_doc_final.tsv")
# Question id and Question text
queries = load_questions_to_df(data_path+"FiQA_train_question_final.tsv")
# Question id and Answer id pair
qid_docid = load_qid_docid_to_df(data_path+"FiQA_train_question_doc_final.tsv")

In [9]:
fiqa_index = "/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/retriever/lucene-index-fiqa/"


def split_label(qid_docid):
    """
    Split question answer pairs into train, test, validation sets.

    Returns:
        train_label: Dictonary
            key - question id
            value - list of relevant docids
        test_label: Dictonary
            key - question id
            value - list of relevant docids
        valid_label: Dictonary
            key - question id
            value - list of relevant docids
    ----------
    Arguments:
        qid_docid: Dataframe containing the question id and relevant docids
    """
    # Group the answers for each question into a list
    qid_docid = qid_docid.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()
    # Split data
    train, test_set = train_test_split(qid_docid, test_size=0.05)
    train_set, valid_set = train_test_split(train, test_size=0.1)
    # Expand the list of docids into individual rows to represent a single sample
    train_data = train_set.explode('docid')
    test_data = test_set.explode('docid')
    valid_data = valid_set.explode('docid')

    # Convert data into dictionary - key: qid, value: list of relevant docid
    train_label = label_to_dict(train_data)
    test_label = label_to_dict(test_data)
    valid_label = label_to_dict(valid_data)

    return train_label, test_label, valid_label

def split_question(train_label, test_label, valid_label, queries):
    """
    Split questions into train, test, validation sets.

    Returns:
        train_questions: Dataframe with qids
        test_questions: Dataframe with qids
        valid_questions: Dataframe with qids
    ----------
    Arguments:
        train_label: Dictionary contraining qid and list of relevant docid
        test_label: Dictionary contraining qid and list of relevant docid
        valid_label: Dictionary contraining qid and list of relevant docid
        queries: Dataframe containing the question id and question text
    """
    # Get a list of question ids
    train_q = list(train_label.keys())
    test_q = list(test_label.keys())
    valid_q = list(valid_label.keys())

    # Split question dataframe into train, test, valid set
    train_questions = queries[queries['qid'].isin(train_q)]
    test_questions = queries[queries['qid'].isin(test_q)]
    valid_questions = queries[queries['qid'].isin(valid_q)]

    return train_questions, test_questions, valid_questions

def split_label(qid_docid):
    """
    Split question answer pairs into train, test, validation sets.

    Returns:
        train_label: Dictonary
            key - question id
            value - list of relevant docids
        test_label: Dictonary
            key - question id
            value - list of relevant docids
        valid_label: Dictonary
            key - question id
            value - list of relevant docids
    ----------
    Arguments:
        qid_docid: Dataframe containing the question id and relevant docids
    """
    # Group the answers for each question into a list
    qid_docid = qid_docid.groupby(['qid']).agg(lambda x: tuple(x)).applymap(list).reset_index()
    # Split data
    train, test_set = train_test_split(qid_docid, test_size=0.05)
    train_set, valid_set = train_test_split(train, test_size=0.1)
    # Expand the list of docids into individual rows to represent a single sample
    train_data = train_set.explode('docid')
    test_data = test_set.explode('docid')
    valid_data = valid_set.explode('docid')

    # Convert data into dictionary - key: qid, value: list of relevant docid
    train_label = label_to_dict(train_data)
    test_label = label_to_dict(test_data)
    valid_label = label_to_dict(valid_data)

    return train_label, test_label, valid_label

def split_question(train_label, test_label, valid_label, queries):
    """
    Split questions into train, test, validation sets.

    Returns:
        train_questions: Dataframe with qids
        test_questions: Dataframe with qids
        valid_questions: Dataframe with qids
    ----------
    Arguments:
        train_label: Dictionary contraining qid and list of relevant docid
        test_label: Dictionary contraining qid and list of relevant docid
        valid_label: Dictionary contraining qid and list of relevant docid
        queries: Dataframe containing the question id and question text
    """
    # Get a list of question ids
    train_q = list(train_label.keys())
    test_q = list(test_label.keys())
    valid_q = list(valid_label.keys())

    # Split question dataframe into train, test, valid set
    train_questions = queries[queries['qid'].isin(train_q)]
    test_questions = queries[queries['qid'].isin(test_q)]
    valid_questions = queries[queries['qid'].isin(valid_q)]

    return train_questions, test_questions, valid_questions

def create_dataset(question_df, labels, cands_size):
    """Retrieves the top-k candidate answers for a question and
    creates a list of lists of the dataset containing the question id,
    list of relevant answer ids, and the list of answer candidates

    Returns:
        dataset: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        question_df: Dataframe containing the qid and question text
        labels: Dictonary containing the qid to text map
        cands_size: int - number of candidates to retrieve
    """
    dataset = []
    # Calls retriever
    searcher = SimpleSearcher(fiqa_index)
    # For each question
    for i, row in question_df.iterrows():
        qid = row['qid']
        tmp = []
        # Append qid
        tmp.append(qid)
        # Append list of relevant docs
        tmp.append(labels[qid])
        # Retrieves answer candidates
        cands = []
        query = row['question']
        query = re.sub('[£€§]', '', query)
        hits = searcher.search(query, k=cands_size)

        for docid in range(0, len(hits)):
            cands.append(int(hits[docid].docid))
        # Append candidate answers
        tmp.append(cands)
        dataset.append(tmp)

    return dataset

def get_dataset(query_path, labels_path, cands_size):
    """Splits the dataset into train, validation, and test set and creates
    the dataset form for training, validation, and testing.

    Returns:
        train_set: list of list in the form [qid, [pos ans], [ans candidates]]
        valid_set: list of list in the form [qid, [pos ans], [ans candidates]]
        test_set: list of list in the form [qid, [pos ans], [ans candidates]]
    ----------
    Arguments:
        query_path: str - path containing a list of qid and questions
        labels_path: str - path containing a list of qid and relevant docid
        cands_size: int - number of candidates to retrieve
    """
    # Question id and Question text
    queries = load_questions_to_df(query_path)
    # Question id and Answer id pair
    qid_docid = load_qid_docid_to_df(labels_path)
    # qid to docid label map
    labels = label_to_dict(qid_docid)
    train_label, test_label, valid_label = split_label(qid_docid)
    # Split Questions
    train_questions, test_questions, \
    valid_questions = split_question(train_label, test_label, valid_label, queries)

    print("\nGenerating training set...\n")
    train_set = create_dataset(train_questions, labels, cands_size)
    print("Generating validation set...\n")
    valid_set = create_dataset(valid_questions, labels, cands_size)
    print("Generating test set...\n")
    test_set = create_dataset(test_questions, labels, cands_size)

    return train_set, valid_set, test_set

def main():

    parser = argparse.ArgumentParser()
    # Required parameters
    parser.add_argument("--query_path", default=None, type=str, required=True,
    help="Path to the question id to text data in .tsv format. Each line should have at least two columns named (qid, question) separated by tab")
    parser.add_argument("--label_path", default=None, type=str, required=True,
    help="Path to the question id and answer id data in .tsv format. Each line should have at two columns named (qid, docid) separated by tab")

    # Optional parameters
    parser.add_argument("--cands_size", default=50, type=int, required=False,
    help="Number of candidates to retrieve per question.")
    parser.add_argument("--output_dir", default=Path.cwd()/'data/data_pickle/',
    type=str, required=False, help="The output directory where the generated data will be stored.")

    args = parser.parse_args()

    if len(sys.argv) < 4:
        print("Usage: python3 src/generate_data.py <query_path> <label_path>")
        sys.exit()

    train_set, valid_set, test_set = get_dataset(args.query_path, \
                                                 args.label_path, \
                                                 args.cands_size)

    save_pickle(args.output_dir + "train_set.pickle", train_set)
    save_pickle(args.output_dir + "valid_set.pickle", valid_set)
    save_pickle(args.output_dir + "test_set.pickle", test_set)

In [10]:
query_path = "/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/FiQA_train_question_final.tsv"
labels_path = "/gdrive/MyDrive/nlp-data/nlp-qa-datasets/FiQA/FiQA_train_task2/FiQA_train_question_doc_final.tsv"
train_set, valid_set, test_set = get_dataset(query_path, labels_path, 50)


Generating training set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
Generating validation set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
Generating test set...

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [11]:
# Cleaning data
empty_docs, empty_id = get_empty_docs(collection)
# Remove empty answers from collection of answers
collection_cleaned = collection.drop(empty_id)
# Remove empty answers from qa pairs
qid_docid = qid_docid[~qid_docid['docid'].isin(empty_docs)]

print("Number of answers after cleaning: {}".format(len(collection_cleaned)))
print("Number of QA pairs after cleaning: {}".format(len(qid_docid)))

Number of answers after cleaning: 57600
Number of QA pairs after cleaning: 17072


In [12]:
processed_answers = process_answers(collection_cleaned)
processed_questions = process_questions(queries)

In [13]:
word2index, word2count = create_vocab(processed_answers, processed_questions)

print("Vocab size: {}".format(len(word2index)))
print("Top {} common words: {}".format(35, Counter(word2count).most_common(35)))

Vocab size: 85034
Top 35 common words: [('the', 371203), ('to', 233559), ('a', 201620), ('you', 166702), ('and', 163066), ('of', 157574), ('is', 129894), ('in', 120019), ('that', 111416), ('for', 89366), ('it', 83822), ('i', 74100), ('your', 68153), ('are', 67255), ('if', 60689), ('be', 59266), ('on', 58382), ('have', 55754), ('as', 50088), ('this', 49868), ('not', 49227), ('or', 46080), ('with', 45894), ('they', 44485), ('but', 41690), ('can', 38863), ('will', 36865), ('at', 35548), ('an', 31392), ('money', 31003), ('so', 29980), ('$', 29096), ('would', 28750), ('from', 28582), ('more', 27378)]


In [14]:
qid_to_text, docid_to_text = id_to_text(collection, queries)
qid_to_tokenized_text, docid_to_tokenized_text = id_to_tokenized_text(processed_answers, processed_questions)

In [15]:
id1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/transient/' + "train_input_ids1.pickle")
type1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/transient/' + "train_token_type_ids1.pickle")
mask1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/transient/' + "train_att_masks1.pickle")
label1 = load_pickle('/gdrive/MyDrive/nlp-yuan_code/FinBERT-QA/data/data_pickle/transient/' + "train_labels1.pickle")
print(len(id1), len(type1), len(mask1), len(label1))

49979 49979 49979 49979


In [16]:
!pip install -q tensorflow-recommenders
!pip install -q scann

In [17]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
#import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [18]:
embedding_dimension = 32

In [19]:
def generate_ids(data):
  qids = []
  aids = []
  qa_pairs = []


  #for i, seq in enumerate(tqdm(train_set)):
  for i, seq in enumerate(data):
    qid, ans_labels, cands = seq[0], seq[1], seq[2]
    #print (qid, ans_labels, cands)
    

    # For each answer in the candidates
    for docid in ans_labels:
      qids.append(qid)
      aids.append(docid)
      qa_pairs.append((qid, docid))

  unique_qids = np.unique(qids)
  unique_aids = np.unique(aids)
  return unique_qids, unique_aids, qa_pairs
      
qids, aids, qa_pairs = generate_ids(train_set)

In [20]:
print(len(qids), len(aids), len(qa_pairs))
type(qids)

5683 14661 14661


numpy.ndarray

In [31]:
unique_questions = np.unique([qid_to_text[qid] for qid in qids])


question_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_questions, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(qids) + 1, embedding_dimension)
])

In [32]:
unique_answers = np.unique([docid_to_text[aid] for aid in aids])

answer_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_answers, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(aids) + 1, embedding_dimension)
])

In [35]:
answers = tf.data.Dataset.from_tensor_slices([docid_to_text[aid] if isinstance(docid_to_text.get(aid),str) else '' for aid in aids])

metrics = tfrs.metrics.FactorizedTopK(
  candidates=answers.batch(128).map(answer_model)
)
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [48]:
questions = tf.data.Dataset.from_tensor_slices([qid_to_text[qid] if isinstance(qid_to_text.get(qid),str) else '' for qid, aid in qa_pairs])
answers = tf.data.Dataset.from_tensor_slices([docid_to_text[aid] if isinstance(docid_to_text.get(aid),str) else '' for qid, aid in qa_pairs])


ds = tf.data.Dataset.zip((questions, answers))
ds = ds.map(lambda x, y : {"question": x, "answer": y})

tf.random.set_seed(42)
shuffled = ds.shuffle(14_661, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(12_000)
test = shuffled.skip(12_000).take(2_661)

In [49]:
len(train), len(test)

(12000, 2661)

In [50]:
class QAModel(tfrs.Model):

  def __init__(self, question_model, answer_model):
    super().__init__()
    self.question_model: tf.keras.Model = question_model
    self.answer_model: tf.keras.Model = answer_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    q_embeddings = self.question_model(features["question"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    a_embeddings = self.answer_model(features["answer"])

    # The task computes the loss and the metrics.
    return self.task(q_embeddings, a_embeddings)

In [52]:
cached_train = train.shuffle(12000).batch(1000).cache()

In [53]:
model = QAModel(question_model, answer_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))


model.fit(cached_train, epochs=3)

Epoch 1/3
12/12 [==============================] - 22s 2s/step - factorized_top_k/top_1_categorical_accuracy: 8.3333e-04 - factorized_top_k/top_5_categorical_accuracy: 0.6845 - factorized_top_k/top_10_categorical_accuracy: 0.8884 - factorized_top_k/top_50_categorical_accuracy: 0.9859 - factorized_top_k/top_100_categorical_accuracy: 0.9983 - loss: 2334.0143 - regularization_loss: 0.0000e+00 - total_loss: 2334.0143
Epoch 2/3
12/12 [==============================] - 21s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0097 - factorized_top_k/top_5_categorical_accuracy: 0.8293 - factorized_top_k/top_10_categorical_accuracy: 0.9544 - factorized_top_k/top_50_categorical_accuracy: 0.9992 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1669.8356 - regularization_loss: 0.0000e+00 - total_loss: 1669.8356
Epoch 3/3
12/12 [==============================] - 21s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0259 - factorized_top_k/top_5_categorical_accuracy: 0.8726

In [54]:
cached_test = test.batch(2661).cache()
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 5s 5s/step - factorized_top_k/top_1_categorical_accuracy: 0.0105 - factorized_top_k/top_5_categorical_accuracy: 0.6077 - factorized_top_k/top_10_categorical_accuracy: 0.8207 - factorized_top_k/top_50_categorical_accuracy: 0.9309 - factorized_top_k/top_100_categorical_accuracy: 0.9733 - loss: 7191.8521 - regularization_loss: 0.0000e+00 - total_loss: 7191.8521


{'factorized_top_k/top_100_categorical_accuracy': 0.9733182787895203,
 'factorized_top_k/top_10_categorical_accuracy': 0.8207440972328186,
 'factorized_top_k/top_1_categorical_accuracy': 0.010522359982132912,
 'factorized_top_k/top_50_categorical_accuracy': 0.9308530688285828,
 'factorized_top_k/top_5_categorical_accuracy': 0.6076663136482239,
 'loss': 7191.85205078125,
 'regularization_loss': 0,
 'total_loss': 7191.85205078125}

In [61]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.question_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((answers.batch(100), answers.batch(100).map(model.answer_model)))
)

In [66]:
# Get recommendations.
_, cands = scann_index(tf.constant([qid_to_text[5]]))

print(f'QUESTION: ', qid_to_text[5])
print(f"Recommendations for user 42: {cands[0, :10]}")


QUESTION:  Starting a new online business
Recommendations for user 42: [b"This could be a case of the new chip card technology and dealing with slow reimbursement turnaround time. I recently visited a restaurant who was not using the chip technology, and it refused my card after several attempts. I found out from my bank it was because the restaurant was not set up for chip and I had not eaten there before....I know at the other end it takes far longer for the funds to get to the merchant; banks don't want to part with other people's money."
 b'Why would such a large discount make business sense to the restaurant? The legit reasons could be;   Or can I assume that the restaurant is trying to avoid leaving a paper trail so that they could avoid paying tax? The illegal reasons could be;'
 b'To expand on keshlam\'s answer: A direct feed does not involve a website of any kind.  Each exchange publishes its order/trade feed(s) onto a packet network where subscribers have machines listening a

In [ ]:
import math
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
def generate_bert_params(ids, m):
  input_ids = []
  token_type_ids = []
  att_masks = []

  for id in ids:
    if m.get(id) and isinstance(m.get(id),str):
      encoded_seq = bert_tokenizer.encode_plus(m[id][:500],
                                              max_length=512,
                                              pad_to_max_length=True,
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_attention_mask = True)
      input_ids.append(encoded_seq['input_ids'])
      token_type_ids.append(encoded_seq['token_type_ids'])
      att_masks.append(encoded_seq['attention_mask'])
  return input_ids, token_type_ids, att_masks

In [ ]:
qp_ids, qp_types, qp_masks = generate_bert_params(qids, qid_to_text)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
ap_ids, ap_types, ap_masks = generate_bert_params(aids, docid_to_text)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(len(ap_ids), len(ap_types), len(ap_masks), len(qp_ids), len(qp_types), len(qp_masks))

14581 14581 14581 5683 5683 5683


In [ ]:
embedding_dimension = 512

In [ ]:
def create_bert_model():

    bert_model = TFBertModel.from_pretrained('bert-base-cased')
    max_length = 512

    #restrict training to the train_layers outer transformer layers
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer') #--SOLUTION--
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')


    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}         

    x = bert_model(bert_inputs)[0][:,0,:]
    return tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs = x)

In [ ]:
q_model = create_bert_model()([qp_ids, qp_types, qp_masks])

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
q_model[0]

NameError: ignored